In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [75]:
import os, json

BASE_PATH = "/content/gdrive/MyDrive/CLAUSEAI/Milestone_1/results/agents"

def load_agent(agent):
    path = f"{BASE_PATH}/{agent}/{agent}_agent_output.json"
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

legal_agent = load_agent("legal")
compliance_agent = load_agent("compliance")
finance_agent = load_agent("finance")
operations_agent = load_agent("operations")

print("Agents loaded:")
print("Legal:", len(legal_agent["output"]["extracted_clauses"]))
print("Compliance:", len(compliance_agent["output"]["extracted_clauses"]))
print("Finance:", len(finance_agent["output"]["extracted_clauses"]))
print("Operations:", len(operations_agent["output"]["extracted_clauses"]))


Agents loaded:
Legal: 2
Compliance: 1
Finance: 3
Operations: 2


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

import json

BASE = "/content/gdrive/MyDrive/CLAUSEAI/Milestone_1/results/agents"

def load_agent(agent):
    with open(f"{BASE}/{agent}/{agent}_agent_output.json", "r") as f:
        data = json.load(f)
    return data["output"]   # <-- unwrap here

legal_output = load_agent("legal")
compliance_output = load_agent("compliance")
finance_output = load_agent("finance")
operations_output = load_agent("operations")

print("Legal clauses:", len(legal_output["extracted_clauses"]))
print("Compliance clauses:", len(compliance_output["extracted_clauses"]))
print("Finance clauses:", len(finance_output["extracted_clauses"]))
print("Operations clauses:", len(operations_output["extracted_clauses"]))


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Legal clauses: 2
Compliance clauses: 1
Finance clauses: 3
Operations clauses: 2


In [ ]:
ROUTING_RULES = {
    "legal": ["termination", "jurisdiction", "indemnity", "governing law"],
    "compliance": ["gdpr", "audit", "regulatory", "data protection"],
    "finance": ["payment", "penalty", "invoice", "fee"],
    "operations": ["sla", "timeline", "deliverable", "milestone"]
}

def route_query(query: str):
    query = query.lower()
    matched = []

    for agent, keywords in ROUTING_RULES.items():
        if any(k in query for k in keywords):
            matched.append(agent)

    return matched


In [ ]:
!pip install -q langgraph


In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Dict


In [ ]:
class GraphState(TypedDict):
    query: str
    memory: List[dict]
    legal: Dict
    compliance: Dict
    finance: Dict
    operations: Dict

In [ ]:
def legal_node(state: GraphState):
    print("▶️ Running LEGAL agent")
    state["legal"] = legal_output
    return state

def compliance_node(state: GraphState):
    print("▶️ Running COMPLIANCE agent")
    state["compliance"] = compliance_output
    return state

def finance_node(state: GraphState):
    print("▶️ Running FINANCE agent")
    state["finance"] = finance_output
    return state

def operations_node(state: GraphState):
    print("▶️ Running OPERATIONS agent")
    state["operations"] = operations_output
    return state


In [ ]:
graph = StateGraph(GraphState)

graph.add_node("legal_agent", legal_node)
graph.add_node("compliance_agent", compliance_node)
graph.add_node("finance_agent", finance_node)
graph.add_node("operations_agent", operations_node)

graph.set_entry_point("legal_agent")

graph.add_edge("legal_agent", "compliance_agent")
graph.add_edge("compliance_agent", "finance_agent")
graph.add_edge("finance_agent", "operations_agent")
graph.add_edge("operations_agent", END)

app = graph.compile()


In [ ]:
input_state = {
    "query": "Review termination, GDPR compliance, payment terms, and SLAs",
    "legal": {},
    "compliance": {},
    "finance": {},
    "operations": {}
}

result = app.invoke(input_state)
result.keys()


▶️ Running LEGAL agent
▶️ Running COMPLIANCE agent
▶️ Running FINANCE agent
▶️ Running OPERATIONS agent


dict_keys(['query', 'legal', 'compliance', 'finance', 'operations'])

In [ ]:
graph = StateGraph(GraphState)

graph.add_node("legal_agent", legal_node)
graph.add_node("compliance_agent", compliance_node)
graph.add_node("finance_agent", finance_node)
graph.add_node("operations_agent", operations_node)

graph.set_entry_point("compliance_agent")

graph.add_edge("compliance_agent", "legal_agent")
graph.add_edge("legal_agent", "finance_agent")
graph.add_edge("finance_agent", "operations_agent")
graph.add_edge("operations_agent", END)

app = graph.compile()

result = app.invoke(input_state)


▶️ Running COMPLIANCE agent
▶️ Running LEGAL agent
▶️ Running FINANCE agent
▶️ Running OPERATIONS agent


In [ ]:
graph = StateGraph(GraphState)

graph.add_node("legal_agent", legal_node)
graph.add_node("compliance_agent", compliance_node)
graph.add_node("operations_agent", operations_node)

graph.set_entry_point("legal_agent")

graph.add_edge("legal_agent", "compliance_agent")
graph.add_edge("compliance_agent", "operations_agent")
graph.add_edge("operations_agent", END)

app = graph.compile()

result = app.invoke(input_state)
result.keys()


▶️ Running LEGAL agent
▶️ Running COMPLIANCE agent
▶️ Running OPERATIONS agent


dict_keys(['query', 'legal', 'compliance', 'finance', 'operations'])

In [ ]:
def route_decision(state: GraphState):
    agents = route_query(state["query"])

    if "legal" in agents:
        return "legal_agent"
    if "compliance" in agents:
        return "compliance_agent"
    if "finance" in agents:
        return "finance_agent"
    if "operations" in agents:
        return "operations_agent"

    return END


In [ ]:
def router_node(state: GraphState):
    print("▶️ Routing query:", state["query"])
    return {
        "query": state["query"],
        "memory": state["memory"],   # 🔥 this was missing
        "legal": state["legal"],
        "compliance": state["compliance"],
        "finance": state["finance"],
        "operations": state["operations"],
    }


In [ ]:
def route_next_agent(state: GraphState):
    query = state["query"].lower()

    if any(k in query for k in ["termination", "jurisdiction", "indemnity"]):
        return "legal"
    if any(k in query for k in ["gdpr", "audit", "regulatory", "data protection"]):
        return "compliance"
    if any(k in query for k in ["payment", "penalty", "invoice", "fee"]):
        return "finance"
    if any(k in query for k in ["sla", "timeline", "deliverable", "milestone"]):
        return "operations"

    return END


In [ ]:
graph = StateGraph(GraphState)

graph.add_node("router", router_node)
graph.add_node("legal", legal_node)
graph.add_node("compliance", compliance_node)
graph.add_node("finance", finance_node)
graph.add_node("operations", operations_node)

graph.set_entry_point("router")

graph.add_conditional_edges(
    "router",
    route_next_agent,
    {
        "legal": "legal",
        "compliance": "compliance",
        "finance": "finance",
        "operations": "operations",
        END: END
    }
)

graph.add_edge("legal", END)
graph.add_edge("compliance", END)
graph.add_edge("finance", END)
graph.add_edge("operations", END)

app = graph.compile()


In [ ]:
state = {
    "query": "Review termination clause",
    "legal": {},
    "compliance": {},
    "finance": {},
    "operations": {}
}
app.invoke(state)


▶️ Routing query: Review termination clause
▶️ Running LEGAL agent


{'query': 'Review termination clause',
 'legal': {'clause_type': 'Legal Analysis',
  'extracted_clauses': ["The other party asserts any rights in or to the terminating party's intellectual property in violation of this Agreement.",
   'The other party shall give notice of termination in writing to the other party, which notice shall specify in reasonable detail the event(s) of default that give rise to such termination.'],
  'risk_level': 'low',
  'confidence': 0.85,
  'evidence': ['The first sentence of the clause describes different events that can lead to termination.']},
 'compliance': {},
 'finance': {},
 'operations': {}}

In [ ]:
state = {
    "query": "Check late payment penalties",
    "legal": {},
    "compliance": {},
    "finance": {},
    "operations": {}
}
app.invoke(state)


▶️ Routing query: Check late payment penalties
▶️ Running FINANCE agent


{'query': 'Check late payment penalties',
 'legal': {},
 'compliance': {},
 'finance': {'clause_type': 'Finance Analysis',
  'extracted_clauses': ['In the event that Provider incurs reasonable and documented out-of-pocket expenses in the provision of any Service, including, without limitation, license fees and payments to third-party service providers or subcontractors (such included expenses, collectively, ',
   '), Recipient shall reimburse Provider for all such Out-of-Pocket Costs.',
   'Provider shall provide Recipient with monthly invo'],
  'risk_level': 'medium',
  'confidence': 0.7,
  'evidence': ['The clause clearly defines the scope of Out-of-Pocket Costs and requires Recipient to reimburse Provider for such expenses.']},
 'operations': {}}

In [ ]:
from typing import List

class GraphState(TypedDict):
    query: str
    memory: List[dict]
    legal: dict
    compliance: dict
    finance: dict
    operations: dict


In [ ]:
def legal_node(state: GraphState):
    print("▶️ Running LEGAL agent")

    state["legal"] = legal_output

    clauses = extract_clauses(legal_output)

    state["memory"].append({
        "agent": "legal",
        "findings": clauses
    })

    return state




def compliance_node(state: GraphState):
    print("▶️ Running COMPLIANCE agent")

    state["compliance"] = compliance_output

    clauses = extract_clauses(compliance_output)

    state["memory"].append({
        "agent": "compliance",
        "findings": clauses
    })

    return state





def finance_node(state: GraphState):
    print("▶️ Running FINANCE agent")
    output = finance_output
    state["finance"] = output
    state["memory"].append({
        "agent": "finance",
        "findings": output.get("extracted_clauses", [])
    })
    return state


def operations_node(state: GraphState):
    print("▶️ Running OPERATIONS agent")
    output = operations_output
    state["operations"] = output
    state["memory"].append({
        "agent": "operations",
        "findings": output.get("extracted_clauses", [])
    })
    return state



In [ ]:
state = {
    "query": "Review GDPR compliance and payment penalties",
    "memory": [],
    "legal": {},
    "compliance": {},
    "finance": {},
    "operations": {}
}

result = app.invoke(state)

print("Executed agents:", result.keys())
print("\nMemory log:")
for m in result["memory"]:
    print(m["agent"], "→", len(m["findings"]), "clauses")


▶️ Routing query: Review GDPR compliance and payment penalties
▶️ Running COMPLIANCE agent
Executed agents: dict_keys(['query', 'memory', 'legal', 'compliance', 'finance', 'operations'])

Memory log:
compliance → 1 clauses


In [ ]:
def finance_node(state: GraphState):
    print("▶️ Running FINANCE agent")

    compliance_findings = [m for m in state["memory"] if m["agent"] == "compliance"]
    print("▶️ FINANCE reading compliance:", compliance_findings)

    state["finance"] = finance_output

    clauses = extract_clauses(finance_output)

    if compliance_findings:
        state["validation_notes"].append(
            "Finance reviewed compliance findings for penalty conflicts."
        )

    state["memory"].append({
        "agent": "finance",
        "findings": clauses
    })

    return state


In [ ]:
graph = StateGraph(GraphState)

graph.add_node("compliance_agent", compliance_node)
graph.add_node("finance_agent", finance_node)
graph.add_node("legal_agent", legal_node)

graph.set_entry_point("compliance_agent")
graph.add_edge("compliance_agent", "finance_agent")
graph.add_edge("finance_agent", "legal_agent")
graph.add_edge("legal_agent", END)

app = graph.compile()


In [ ]:
def extract_clauses(agent_output):
    # Works for both wrapped and flat outputs
    if "output" in agent_output:
        return agent_output["output"].get("extracted_clauses", [])
    return agent_output.get("extracted_clauses", [])


def compliance_node(state):
    print("▶️ Running COMPLIANCE agent")
    state["compliance"] = compliance_output

    clauses = extract_clauses(compliance_output)

    state["memory"].append({
        "agent": "compliance",
        "findings": clauses
    })
    return state


def finance_node(state):
    print("▶️ Running FINANCE agent")

    compliance_findings = [m for m in state["memory"] if m["agent"] == "compliance"]
    print("▶️ FINANCE reading compliance:", compliance_findings)

    state["finance"] = finance_output

    clauses = extract_clauses(finance_output)

    if compliance_findings:
        state["validation_notes"].append(
            "Finance reviewed compliance findings for penalty conflicts."
        )

    state["memory"].append({
        "agent": "finance",
        "findings": clauses
    })
    return state


def legal_node(state):
    print("▶️ Running LEGAL agent")
    state["legal"] = legal_output

    clauses = extract_clauses(legal_output)

    state["memory"].append({
        "agent": "legal",
        "findings": clauses
    })
    return state


In [ ]:
from langgraph.graph import StateGraph, END

class GraphState(dict):
    pass

graph = StateGraph(GraphState)

graph.add_node("compliance", compliance_node)
graph.add_node("finance", finance_node)
graph.add_node("legal", legal_node)

graph.set_entry_point("compliance")
graph.add_edge("compliance", "finance")
graph.add_edge("finance", "legal")
graph.add_edge("legal", END)

app = graph.compile()


In [ ]:
from typing import TypedDict, List, Dict

class GraphState(TypedDict):
    query: str
    memory: List[dict]
    validation_notes: List[str]
    legal: dict
    compliance: dict
    finance: dict
    operations: dict


In [ ]:
def ensure_state_lists(state):
    if "memory" not in state:
        state["memory"] = []
    if "validation_notes" not in state:
        state["validation_notes"] = []


def compliance_node(state):
    ensure_state_lists(state)

    print("▶️ Running COMPLIANCE agent")
    state["compliance"] = compliance_output

    clauses = extract_clauses(compliance_output)

    state["memory"].append({
        "agent": "compliance",
        "findings": clauses
    })
    return state


def finance_node(state):
    ensure_state_lists(state)

    print("▶️ Running FINANCE agent")
    compliance_findings = [m for m in state["memory"] if m["agent"] == "compliance"]
    print("▶️ FINANCE reading compliance:", compliance_findings)

    state["finance"] = finance_output
    clauses = extract_clauses(finance_output)

    if compliance_findings:
        state["validation_notes"].append(
            "Finance reviewed compliance findings for penalty conflicts."
        )

    state["memory"].append({
        "agent": "finance",
        "findings": clauses
    })
    return state


def legal_node(state):
    ensure_state_lists(state)

    print("▶️ Running LEGAL agent")
    state["legal"] = legal_output

    clauses = extract_clauses(legal_output)

    state["memory"].append({
        "agent": "legal",
        "findings": clauses
    })
    return state


In [ ]:
from langgraph.graph import StateGraph, END

graph = StateGraph(GraphState)

graph.add_node("compliance", compliance_node)
graph.add_node("finance", finance_node)
graph.add_node("legal", legal_node)

graph.set_entry_point("compliance")
graph.add_edge("compliance", "finance")
graph.add_edge("finance", "legal")
graph.add_edge("legal", END)

app = graph.compile()


In [ ]:
state = {
    "query": "Review GDPR compliance and payment penalties",
    "memory": [],
    "validation_notes": [],
    "legal": {},
    "compliance": {},
    "finance": {},
    "operations": {}
}

result = app.invoke(state)

print("\nExecuted agents:", result.keys())

print("\nMemory log:")
for m in result["memory"]:
    print(m["agent"], "→", len(m["findings"]), "clauses")

print("\nValidation notes:")
print(result["validation_notes"])


▶️ Running COMPLIANCE agent
▶️ Running FINANCE agent
▶️ FINANCE reading compliance: [{'agent': 'compliance', 'findings': ["The receiving party will not disclose the other party's confidential information to any third parties without the other party's prior written consent."]}]
▶️ Running LEGAL agent

Executed agents: dict_keys(['query', 'memory', 'validation_notes', 'legal', 'compliance', 'finance', 'operations'])

Memory log:
compliance → 1 clauses
finance → 3 clauses
legal → 2 clauses

Validation notes:
['Finance reviewed compliance findings for penalty conflicts.']


In [64]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [68]:
!pip install -q pinecone


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 745.9/745.9 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.9/280.9 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.4 MB/s eta 0:00:00


In [69]:
import os
from pinecone import Pinecone

os.environ["PINECONE_API_KEY"] = "pcsk_uBxTV_Gibjn6KjaVSDyT5ipaLe7a1kstm7cgmA5SKD9nQcRBv97ws74BF92woYM7WJ8Jt"   # same key you used in Milestone-1

pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])
index = pc.Index("cuad-index")


In [70]:
import json

BASE_PATH = "/content/gdrive/MyDrive/CLAUSEAI/Milestone_1"
CHUNKS_PATH = BASE_PATH + "/Data/chunks"

def get_chunk_text(contract_id, chunk_id):
    fname = f"{contract_id}_chunks.json"
    with open(f"{CHUNKS_PATH}/{fname}") as f:
        chunks = json.load(f)
    for c in chunks:
        if c["chunk_id"] == chunk_id:
            return c["text"]
    return None


In [71]:
def embed_query(query):
    return model.encode([query])[0].tolist()


In [72]:
def rag_search(query, top_k=8):
    query_vec = embed_query(query)

    results = index.query(
        vector=query_vec,
        top_k=top_k,
        include_metadata=True
    )

    matches = []
    for m in results["matches"]:
        cid = m["metadata"]["contract_id"]
        chunk_id = m["metadata"]["chunk_id"]
        text = get_chunk_text(cid, chunk_id)

        matches.append({
            "id": m["id"],
            "score": m["score"],
            "contract_id": cid,
            "chunk_id": chunk_id,
            "text": text
        })

    return matches


In [73]:
compliance_rag = rag_search(COMPLIANCE_QUERY, top_k=8)


In [79]:
def check_grounding(agent_output, rag_chunks):
    """
    Ensures every extracted clause & evidence exists in RAG text.
    """
    all_text = "\n".join(rag_chunks).lower()

    # Check extracted clauses
    for clause in agent_output.get("extracted_clauses", []):
        if clause.lower() not in all_text:
            return False

    # Check evidence
    for ev in agent_output.get("evidence", []):
        if ev.lower() not in all_text:
            return False

    return True


In [76]:
def run_pipeline(agent_output, rag_chunks):
    grounded = check_grounding(
        agent_output["output"],
        [c["text"] for c in rag_chunks if c.get("text")]
    )

    return {
        "grounded": grounded,
        "analysis": agent_output["output"]
    }


In [80]:
COMPLIANCE_QUERY = """
Identify clauses related to:
- GDPR
- Data protection
- Audits
- Regulatory compliance
"""

compliance_rag = rag_search(COMPLIANCE_QUERY, top_k=8)

compliance_pipeline = {
    "query": COMPLIANCE_QUERY,
    "rag_chunks_used": len(compliance_rag),
    **run_pipeline(compliance_agent, compliance_rag)
}

compliance_pipeline


{'query': '\nIdentify clauses related to:\n- GDPR\n- Data protection\n- Audits\n- Regulatory compliance\n',
 'rag_chunks_used': 8,
 'grounded': False,
 'analysis': {'clause_type': 'Compliance Analysis',
  'extracted_clauses': ["The receiving party will not disclose the other party's confidential information to any third parties without the other party's prior written consent."],
  'risk_level': 'high',
  'confidence': 1.0,
  'evidence': ["The clause clearly requires the receiving party to maintain confidentiality of the other party's confidential information."]}}

In [82]:
LEGAL_QUERY = "termination governing law jurisdiction indemnification"
FINANCE_QUERY = "payment invoice penalties late fees"
OPERATIONS_QUERY = "sla timelines milestones deliverables uptime"

legal_rag = rag_search(LEGAL_QUERY, top_k=8)
finance_rag = rag_search(FINANCE_QUERY, top_k=8)
operations_rag = rag_search(OPERATIONS_QUERY, top_k=8)

legal_pipeline = {
    "query": LEGAL_QUERY,
    "rag_chunks_used": len(legal_rag),
    **run_pipeline(legal_agent, legal_rag)
}

finance_pipeline = {
    "query": FINANCE_QUERY,
    "rag_chunks_used": len(finance_rag),
    **run_pipeline(finance_agent, finance_rag)
}

operations_pipeline = {
    "query": OPERATIONS_QUERY,
    "rag_chunks_used": len(operations_rag),
    **run_pipeline(operations_agent, operations_rag)
}

legal_pipeline, finance_pipeline, operations_pipeline


({'query': 'termination governing law jurisdiction indemnification',
  'rag_chunks_used': 8,
  'grounded': False,
  'analysis': {'clause_type': 'Legal Analysis',
   'extracted_clauses': ["The other party asserts any rights in or to the terminating party's intellectual property in violation of this Agreement.",
    'The other party shall give notice of termination in writing to the other party, which notice shall specify in reasonable detail the event(s) of default that give rise to such termination.'],
   'risk_level': 'low',
   'confidence': 0.85,
   'evidence': ['The first sentence of the clause describes different events that can lead to termination.']}},
 {'query': 'payment invoice penalties late fees',
  'rag_chunks_used': 8,
  'grounded': False,
  'analysis': {'clause_type': 'Finance Analysis',
   'extracted_clauses': ['In the event that Provider incurs reasonable and documented out-of-pocket expenses in the provision of any Service, including, without limitation, license fees an

In [83]:
def coordinator_merge(legal, compliance, finance, operations):
    return {
        "legal": legal["analysis"],
        "compliance": compliance["analysis"],
        "finance": finance["analysis"],
        "operations": operations["analysis"]
    }

merged_output = coordinator_merge(
    legal_pipeline,
    compliance_pipeline,
    finance_pipeline,
    operations_pipeline
)

merged_output.keys()


dict_keys(['legal', 'compliance', 'finance', 'operations'])

In [84]:
risk_map = {"low": 1, "medium": 2, "high": 3}

def overall_risk(merged):
    risks = [v["risk_level"] for v in merged.values() if "risk_level" in v]
    return max(risks, key=lambda r: risk_map.get(r, 0)) if risks else "unknown"

overall_contract_risk = overall_risk(merged_output)
overall_contract_risk


'high'

In [85]:
final_milestone2_output = {
    "overall_risk": overall_contract_risk,
    "legal": legal_pipeline,
    "compliance": compliance_pipeline,
    "finance": finance_pipeline,
    "operations": operations_pipeline
}

final_milestone2_output


{'overall_risk': 'high',
 'legal': {'query': 'termination governing law jurisdiction indemnification',
  'rag_chunks_used': 8,
  'grounded': False,
  'analysis': {'clause_type': 'Legal Analysis',
   'extracted_clauses': ["The other party asserts any rights in or to the terminating party's intellectual property in violation of this Agreement.",
    'The other party shall give notice of termination in writing to the other party, which notice shall specify in reasonable detail the event(s) of default that give rise to such termination.'],
   'risk_level': 'low',
   'confidence': 0.85,
   'evidence': ['The first sentence of the clause describes different events that can lead to termination.']}},
 'compliance': {'query': '\nIdentify clauses related to:\n- GDPR\n- Data protection\n- Audits\n- Regulatory compliance\n',
  'rag_chunks_used': 8,
  'grounded': False,
  'analysis': {'clause_type': 'Compliance Analysis',
   'extracted_clauses': ["The receiving party will not disclose the other part